# Palantir IV Smile – Earnings Event Study

Construct IV smiles for PLTR around earnings, estimate implied vs realized move, and illustrate a short straddle payoff.


In [ ]:
import pandas as pd, numpy as np, yfinance as yf, matplotlib.pyplot as plt
from datetime import datetime
plt.rcParams['figure.dpi'] = 120
ticker = 'PLTR'

## 1) Fetch expiries and a sample chain

In [ ]:
stk = yf.Ticker(ticker)
expiries = stk.options
expiries[:5] if expiries else []

## 2) Build a single-maturity IV smile (prototype)

In [ ]:
hist = stk.history(period='1y')
spot = float(hist['Close'].iloc[-1])
expiry = expiries[0] if expiries else None
chain = stk.option_chain(expiry) if expiry else None
calls = chain.calls if chain else pd.DataFrame()
puts  = chain.puts  if chain else pd.DataFrame()

def prep(df, typ):
    if df is None or df.empty:
        return pd.DataFrame()
    out = df[['contractSymbol','strike','lastPrice','impliedVolatility']].copy()
    out['type'] = typ
    out['moneyness'] = out['strike'] / spot
    out['iv'] = out['impliedVolatility']
    return out

opt = pd.concat([prep(calls,'call'), prep(puts,'put')], ignore_index=True)
opt = opt.dropna(subset=['iv'])
fig, ax = plt.subplots()
for typ, g in opt.groupby('type'):
    ax.scatter(g['moneyness'], g['iv'], s=12, alpha=0.7, label=typ)
ax.axvline(1.0, color='k', linestyle='--', linewidth=1)
ax.set_xlabel('Strike / Spot (K/S)')
ax.set_ylabel('Implied Volatility')
ax.set_title(f'{ticker} IV Smile (expiry: {expiry})')
ax.legend()
plt.tight_layout()
plt.savefig('figures/iv_smile_example.png')
plt.show()

## 3) Extend: pre vs post earnings smiles, implied vs realized move